In [13]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from nltk.classify.scikitlearn import SklearnClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
import numpy as np
from nltk.tokenize import word_tokenize

In [3]:
training_data=pd.read_csv('0000000000002747_training_twitter_x_y_train.csv')
testing_data=pd.read_csv('0000000000002747_test_twitter_x_test.csv')

In [19]:
training_documents=training_data['text'].values
training_categories=training_data['airline_sentiment'].values
testing_documents=testing_data['text'].values

In [24]:
training_documents_tokenized = [word_tokenize(doc) for doc in training_documents]
testing_documents_tokenized = [word_tokenize(doc) for doc in testing_documents]

In [22]:
from nltk.corpus import stopwords
import string

stops = set(stopwords.words('english'))
punctuations = list(string.punctuation)
stops.update(punctuations)

In [27]:
from nltk.corpus import wordnet

def get_simple_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

In [23]:
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag

lemmatizer = WordNetLemmatizer()

def clean_review(words):
    output_words = []
    for w in words:
        if w.lower() not in stops:
            pos = pos_tag([w]) # pass an array and not a single word
            clean_word = lemmatizer.lemmatize(w, pos = get_simple_pos(pos[0][1])) 
            output_words.append(clean_word.lower())
    return output_words

In [28]:
training_documents_clean = [clean_review(doc) for doc in training_documents_tokenized]
testing_documents_clean = [clean_review(doc) for doc in testing_documents_tokenized]

In [30]:
train_documents = [" ".join(doc) for doc in  training_documents_clean]
test_documents = [" ".join(doc) for doc in testing_documents_clean]

In [60]:
count_vect=TfidfVectorizer(max_features=10000, max_df=0.9, min_df=0.001)
x_train=count_vect.fit_transform(train_documents)
x_test=count_vect.transform(test_documents)
y_train=training_categories

In [67]:
# clf=RandomForestClassifier(n_estimators=2000, n_jobs=-1)
# clf.fit(x_train, y_train)
# y_test=clf.predict(x_test)

from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV

# clf = SVC(C=100.0, gamma=0.005)
# grid = {"C":[1e2, 1e3, 5e3, 1e4, 5e4, 1e5], "gamma":[1e-3, 5e-4, 1e-4, 5e-3]}
# abc = GridSearchCV(clf, grid)
# abc.fit(x_train, y_train)

svc = SVC(C=10.0, gamma=0.005)
svc.fit(x_train, y_train)
y_test = svc.predict(x_test)

In [68]:
np.savetxt(fname='twitter_predicted.csv', X=y_test, fmt='%s')